In [1]:
print('hello')


hello


In [31]:
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance,
    VectorParams,
    PayloadSchemaType,
    PointStruct,
)
import pandas as pd
import openai
import os

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Please enter your OpenAI API key: ").strip()
    os.environ["OPENAI_API_KEY"] = api_key
    if not api_key:
        raise ValueError("API key is required.")


In [4]:
df_items = pd.read_json("../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)


In [5]:
df_items.head(2)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,[],4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN


In [6]:
def preprocess_data(row):
    return f"{row['title']} {' '.join(row['features'])}"

In [7]:
df_items.head(2)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,[],4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN


In [9]:
def extract_first_large_image(row):
    return row['images'][0].get('large', '')

In [10]:
df_items["preprocessed_data"] = df_items.apply(preprocess_data, axis=1)
df_items["first_large_image"] = df_items.apply(extract_first_large_image, axis=1)


In [11]:
df_items.head(2)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,preprocessed_data,first_large_image
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",https://m.media-amazon.com/images/I/51G07yWoOB...
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,[],4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",https://m.media-amazon.com/images/I/41bOA5-ogW...


In [12]:
qdrant_client = QdrantClient(url="http://localhost:6333")


In [14]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-01-hybrid",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

True

In [17]:
qdrant_client.create_payload_index(
    collection_name="Amazon-items-collection-01-hybrid",
    field_name="text",
    field_schema=PayloadSchemaType.TEXT
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [21]:
df_sample = df_items.sample(50, random_state=25)

In [19]:
def get_embeddings(text, model="text-embedding-3-small"):
    
    response = openai.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

In [25]:
data_to_embed = df_sample[["preprocessed_data", "first_large_image", "rating_number", "price", "average_rating"]].to_dict(orient="records")

In [26]:
data_to_embed

[{'preprocessed_data': 'Bluetooth Car Adapter, LDNIO Bluetooth FM Transmitter for Car, 43W PD&QC 3.0 Three USB Port Car Bluetooth Adapter with LED Display, Hands-Free Calling, and AUX Input for All Smartphones Audio Player Fast Charging Type C Multi Ports: USB Type C Durable Fast Connect & Clear Bluetooth Sound 3 in 1 Value',
  'first_large_image': 'https://m.media-amazon.com/images/I/41gqwN41WpL._AC_.jpg',
  'rating_number': 110,
  'price': 21.0,
  'average_rating': 3.8},
 {'preprocessed_data': 'Bluetooth Multi-Device Keyboard, Dual Channel Universal Rechargeable Wireless Keyboard with Integrated Stand for iPad Smartphone Tablet MacBook iOS Windows Android Devices - Pink 【Easy-Switch to 2 Devices】 Simply press the FN+BT1/BT2 to switch typing between 2 connected Bluetooth devices, work with your smartphone and tablet on the slot stablely. 【Type Anywhere in Comfort】0.46in thick and 11.34in long, the compact Bluetooth keyboard is small enough to tuck into your briefcase and light enough 

In [32]:
pointstructs = []
for i, data in enumerate(data_to_embed):
    embedding = get_embeddings(data["preprocessed_data"])
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embedding,
            payload={
                "text": data["preprocessed_data"],
                "first_large_image": data["first_large_image"],
                "rating_number": data["rating_number"],
                "price": data["price"],
                "average_rating": data["average_rating"]
            }
        )
    )

In [33]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-01-hybrid",
    wait=True,
    points=pointstructs
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
from qdrant_client.models import Filter, Prefetch, FieldCondition, MatchText, FusionQuery

def retrieve_data(query, k=5):
    query_embedding = get_embeddings(query)
    
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-hybrid",
        prefetch=[
            Prefetch(
                query=query_embedding,
                limit=20,
            ),
            Prefetch(
                filter=Filter(
                    must=[
                        FieldCondition(
                            key="text",
                            match=MatchText(text=query)
                        )
                    ]
                ),
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k
    )
    
    return results

In [36]:
retrieve_data("earphones").points

[ScoredPoint(id=2, version=2, score=1.0, payload={'text': 'Active Noise Cancelling Wireless Earbuds, Bluetooth in-Ear Headphones Built-in 4 Mic ENC Call, Deep Bass Ear Buds,IPX6 Waterproof Stereo Earphones for iPhone,Samsung,Laptop (Blue) ♪【ANC Noise Cancelling&Game Mode】: These premium noise-canceling earbuds adopts professional audio active noise canceling technology, and the binaural microphone blocks environmental noise and enhances the voice experience. M48 wireless earbuds are built to provide the exceptional mobile gaming audio experience with an ultra-low 45ms latency.Taps 5 times to enter game mode,through quick sound effect and ideal gaming audio you never miss any game sound effects. ♪【HiFi Stereo Sound & Superior Clear Call 】: Heaphones equipped with a high-sensitivity diaphragm and dual 13mm drivers to restore audio authenticity and voice dynamics, crystal clarity and deep resonance bass characteristics enable you to enjoy immersive sound quality. Call noise canceling brin

In [37]:
import instructor
from pydantic import BaseModel
from openai import OpenAI


In [ ]:
class RAGGenerationResponse(BaseModel):
    answer: str

client = instructor.from_openai(OpenAI())

prompt = """
You are a helpful assistant.
Return an answer to the question.
Question: What is your name?
"""
response, raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1",
    response_model=RAGGenerationResponse,
    messages=[{"role": "user", "content": prompt}],
    temperature=0.5
)